In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [ ]:
df = pd.read_csv("./data/Input_data_filled.csv")
df.head(3)

,postal_code,price,number_of_bedrooms,living_area,equiped_kitchen,furnished,terrace,garden,swimming_pool,property_type
0,2800,329000,3.0,104,0,0,1,0,0,Apartment_Apartment
1,2200,425000,3.0,378,0,0,1,1,0,House_Residence
2,2840,264700,1.0,69,0,0,1,0,0,Apartment_Apartment


### Data Splitting

In [ ]:
def split_data(df, target, test_size=0.2, random_state=42):
    X = df.drop(columns=[target])
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test
    
df = pd.read_csv("./data/Input_data_filled_encoded.csv")
X_train, X_test, y_train, y_test = split_data(df, target="price")

### Model Evaluation

In [ ]:
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    r2 = r2_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    return {"r2": r2,"mse": mse}

### Model Building + Training + Evaluation

In [ ]:
def build_xgb_model():
    model = XGBRegressor(
    n_estimators=775, # Количество деревьев в ансамбле, чем больше, тем выше качество, но медленнее обучение, типовое значение: 100–1000
    learning_rate=0.0125, # Скорость обучения, Маленькое значение = модель учится медленно, но устойчиво, большое значение = модель быстрее переобучается, Обычно 0.01–0.3
    max_depth=6, # Максимальная глубина каждого дерева, глубокие деревья → модель точнее, но легко переобучается, Мелкие деревья → более обобщающие. Типичный диапазон: 3–10
    subsample=0.8, # Доля строк (обучающих наблюдений), используемая для каждого дерева. 0.8 = каждое дерево обучается на 80% случайно выбранных данных. Помогает бороться с переобучением
    colsample_bytree=0.78, # Доля признаков (features), используемых для построения каждого дерева. 0.8 = использовать 80% случайных колонок на дерево. Помогает снизить переобучение и ускорить обучение
    objective="reg:squarederror",
    random_state=42 # Фиксация случайности. Нужна, чтобы модель каждый раз давала одинаковый результат при одних и тех же данных
    )
    return model

def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

model = build_xgb_model()
model = train_model(model, X_train, y_train)
print(evaluate_model(model, X_test, y_test))

{'r2': 0.7048772573471069, 'mse': 17538379776.0}


In [ ]:
# Model: Saving + Loading
joblib.dump(model, "price_xgb.pkl")
loaded_model = joblib.load("price_xgb.pkl")